In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download("punkt")
nltk.download("stopwords")

# Load qualification data from the "qualification level" Excel file
qualification_data = pd.read_excel("../data/qualification level.xlsx")
qualification_dict = dict(zip(qualification_data["qualification"], qualification_data["mqf level"]))

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalnum()]
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return " ".join(tokens)

def compare_skills(user_skills, sector_skills):
    user_skills = preprocess_text(user_skills)
    sector_skills = preprocess_text(sector_skills)

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([user_skills, sector_skills])
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

    return cosine_sim[0][0]

def main():
    print("Welcome to the Skill Matching Program!")

    user_skills = input("Please enter your skills, separated by commas: ")
    user_qualification = input("Please enter your qualification level (1 to 8): ")
    user_sector = input("Please enter your interested sector (leave blank if not specified): ")

    sectors_data = pd.read_excel("../data/skill by sector.xlsx")
    course_suggestions_data = pd.read_excel("../data/course suggestion.xlsx")
    matching_sectors = []

    for _, row in sectors_data.iterrows():
        sector = row["sector"]
        sector_skills = row["skills"]
        min_qualification = row["qualification"]
        similarity_score = compare_skills(user_skills, sector_skills)

        if similarity_score > 0 and int(user_qualification) >= int(min_qualification):
            if not user_sector or user_sector.lower() in sector.lower():
                matching_sectors.append(sector)

    if matching_sectors:
        print("Matching Sectors:")
        for sector in matching_sectors:
            sector_row = sectors_data.loc[sectors_data["sector"] == sector].iloc[0]
            required_skills = set(sector_row["skills"].split(","))
            user_input_skills = set(user_skills.split(","))
            matching_skills = user_input_skills.intersection(required_skills)
            lacking_skills = required_skills.difference(user_input_skills)

            print(f"Sector: {sector}")
            print("Matching Skills:", ", ".join(matching_skills))
            print("Lacking Skills:", ", ".join(lacking_skills))
            print(f"Minimum Qualification: MQF Level {sector_row['qualification']}")

            # Add course suggestions based on the sector
            course_suggestions = course_suggestions_data.loc[course_suggestions_data['sector'] == sector]
            if not course_suggestions.empty:
                print("Course Suggestions:")
                for _, suggestion_row in course_suggestions.iterrows():
                    for col in course_suggestions.columns[1:]:
                        suggestion = suggestion_row[col]
                        suggestion_link = f"<a href='{suggestion}'>{col}</a>"
                        print(f"- {suggestion_link}")

            if len(lacking_skills) > 0:
                print("Job Description:")
                job_description = sector_row["job description"].split(";")
                for desc in job_description:
                    print(desc.strip())
            else:
                print("Congratulations! You have the required skills for this sector.")
            print()  # Add a blank line after each sector recommendation
    else:
        print("Sorry, no matching sectors found in our database for your skills and qualification level.")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to C:\Users\Khor Kean
[nltk_data]     Teng\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Khor Kean
[nltk_data]     Teng\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Welcome to the Skill Matching Program!
Sorry, no matching sectors found in our database for your skills and qualification level.
